In [5]:
import os
os.chdir("..")

In [14]:
import json
import numpy as np
import math
from typing import Dict, List, Tuple
import copy
from itertools import combinations
from collections import Counter

import logging
from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector, ConflictPredicator

root_folder = os.path.join(DATA_ROOT, THERMO_ROOT)

test_projects = [
    "HS3301",
    # "HS3304",
    # "HS3307",
    # "HS3308",
    "HS3309",
    # "HS3310",
    # "HS3311",
    # "HS3313",
    # "HS5302",
    # "HS5303",
    # "HS5304",
    # "HS5305",
    "HS5309",
    # "HS5314",
]
all_setpoints = [18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5]
setpoint_mapping = {
    "thermostat": {"off": 0, 0: "off"}
}
for idx, point in enumerate(all_setpoints):
    state_full_name = "auto" + DEVICE_MULTI_STATE_SUFFIX + "#" + str(point)
    setpoint_mapping["thermostat"][state_full_name] = idx + 1
    setpoint_mapping["thermostat"][idx + 1] = state_full_name

setpoint_mapping = {
    "thermostat": thermo_state
}
ctx_info = ContextAccessor({
        TIME_CTX: {
            "range" : (0, 24*60),
            "interval" : 60,
        },
        "OutTemp#NUM": {
            "range": (-25, 35),
            "interval": 5,
        },
        THERMO_MODE_CTX: {},
        # WEEKDAY_CTX: {
        #     "range": (0, 6),
        #     "interval": 1,
        # },
})
capacity = {
    "thermostat":0
}
BOOL_SIM = False
BOOL_UMASS= False
ccp_alpha = 0.008
test_dates = generate_test_date(root_folder, test_projects, test_ratio = 0.4, true_random=False, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)

In [15]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], grid_data = test_umass(root_folder, test_project=p, ctx_info=ctx_info, ccp_alpha=ccp_alpha, test_dates=test_dates, is_sim=BOOL_SIM, is_umass=BOOL_UMASS, d_mapping=setpoint_mapping)

c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print("Final predicted conflicts" + str({x:len(final_conflicts[x]) for x in final_conflicts}))

Final predicted conflicts{'thermostat': 25}


In [24]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))
        # for g in d_groups:
        #     print(ctx_info.coor_box_to_range(g["box"][0] + g["box"][1]))
        #     print(g["dis"])

probs_i = [(x["prob"], i) for i, x in enumerate(final_conflicts["thermostat"])]
# print(sorted(probs_i))

# for home, groups in habit_groups.items():
#     for d, d_groups in groups.items():
#         for i, g in enumerate(d_groups):
#             for j in range(i+1, len(d_groups)):
#                 box_1 = g["box"][0] + g["box"][1]
#                 box_2 = d_groups[j]["box"][0] + d_groups[j]["box"][1]
#                 if does_intersect(box_1, box_2):
#                     print("!!!")
            

The habit groups found in HS3301
thermostat 7
The habit groups found in HS3309
thermostat 3
The habit groups found in HS5309
thermostat 6


In [17]:
import copy
# Make ground truth:

gt_ctx_info = ContextAccessor({
        TIME_CTX: {
            "range" : (0, 24*60),
            "interval" : 60,
        },
        "OutTemp#NUM": {
            "range": (-25, 35),
            "interval": 5,
        },
        THERMO_MODE_CTX: {},
        # WEEKDAY_CTX: {
        #     "range": (0, 6),
        #     "interval": 1,
        # },
})
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(root_folder, p, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
    device_events[p] = device_evts
gtconflict_cfg = {
    "context_info": gt_ctx_info,
    "capacity": capacity
}
# test_dates = {}

conflict_finder = GtConflictFinder(gtconflict_cfg)
gt_conflicts, test_state_cnt = conflict_finder.get_Gt_conflict(ctx_evts, device_events, test_dates)
print(len(gt_conflicts))

2607


In [18]:
conflict_device = {
    d:[]
    for d in capacity
}

conflict_state_device = {
    d:{}
    for d in capacity
}
MIN_TEST_OBS = 19
cnts = []
for c in gt_conflicts:
    d = c["device"]
    conflict_device[d].append(c)
    s = gt_ctx_info.get_coor_by_ctx(c["ctx"])
    users = frozenset(c["device_states"].keys())
    cnts.append(test_state_cnt[d][users][s])
    if test_state_cnt[d][users][s] < MIN_TEST_OBS:
        continue
    if s not in conflict_state_device[d]:
        conflict_state_device[d][s] = {}
    conflict_state_device[d][s][users] = conflict_state_device[d][s].get(users, 0) + 1

    # if conflict_state_device[d][s][users] > test_state_cnt[d][s]:
    #     print(conflict_state_device[d][s][users], test_state_cnt[d][s])
    #     print(users, d, c["cur_time"], s)
    
print({d:len(c) for d,c in conflict_device.items()})
print({d:len(c) for d,c in conflict_state_device.items()})
print(len(test_dates))
print(Counter(cnts))

{'thermostat': 2607}
{'thermostat': 47}
145
Counter({6.0: 155, 7.0: 116, 22.0: 114, 26.0: 110, 24.0: 108, 20.0: 102, 8.0: 97, 17.0: 96, 33.0: 89, 30.0: 82, 9.0: 81, 5.0: 81, 27.0: 78, 4.0: 78, 3.0: 78, 13.0: 75, 10.0: 72, 21.0: 69, 11.0: 68, 15.0: 58, 19.0: 57, 12.0: 55, 29.0: 52, 35.0: 51, 14.0: 48, 39.0: 46, 23.0: 44, 37.0: 43, 16.0: 40, 2.0: 39, 25.0: 36, 34.0: 35, 44.0: 34, 28.0: 31, 40.0: 31, 31.0: 27, 47.0: 26, 18.0: 20, 46.0: 18, 42.0: 17, 38.0: 15, 1.0: 14, 36.0: 12, 32.0: 9})


In [19]:
# print(conflict_state_device["thermostat"])

In [20]:
all_users = test_projects
all_devices = capacity.keys()
user_pairs = list(combinations(all_users, 2))
conflict_predicator = ConflictPredicator(ctx_info, final_conflicts)
exp_cnt = 0
exp_gt_c = 0
exp_result = {
    d: {
        frozenset(u_pair): [0,0,0,0,0] 
        for u_pair in user_pairs
    }
    for d in all_devices}
max_p = 0
all_state_cnt = 0
gt_probs = []
for d in all_devices:
    for u_pair in user_pairs:
        u_pair_set = frozenset(u_pair)
        it = np.nditer(test_state_cnt[d][u_pair_set], flags=['multi_index'])
        for count in it:
            all_state_cnt += count
            if count < MIN_TEST_OBS:
                continue
            state = it.multi_index
            gt_prob = 0.
            if (d in conflict_state_device) and \
                (state in conflict_state_device[d]) and \
                (u_pair_set in conflict_state_device[d][state]):

                gt_prob =  float(conflict_state_device[d][state][frozenset(u_pair)]) / count
            gt_ctx_snapshot = gt_ctx_info.coor_to_snapshot(state)

            pred_prob = conflict_predicator.get_prob_conflict(ctx_info.get_coor_by_ctx(gt_ctx_snapshot), u_pair, d)
            exp_cnt += 1
            gt_probs.append(gt_prob)
            if gt_prob > 0:
                # if gt_prob > 0.5:
                #     print(gt_ctx_info.coor_to_snapshot(state).values(), gt_prob, pred_prob, u_pair)
                exp_gt_c += 1
                exp_result[d][u_pair_set][0] += abs(pred_prob - gt_prob)
                exp_result[d][u_pair_set][1] += 1
                exp_result[d][u_pair_set][4] += gt_prob
                if gt_prob > 1:
                    print(gt_prob, pred_prob, d, u_pair)
                # if abs(gt_prob - pred_prob) > 0.2:
                # print(pred_prob, gt_prob, gt_ctx_snapshot, count, u_pair)
                max_p = max(gt_prob, max_p)

            else:
                exp_result[d][u_pair_set][2] += pred_prob - gt_prob
                exp_result[d][u_pair_set][3] += 1
                # print(pred_prob, gt_prob, gt_ctx_snapshot, count, u_pair)

overall_acc = [0.,0.]
for d in exp_result:
    for u_pair in user_pairs:
        u_pair_set = frozenset(u_pair)
        overall_acc[0] += exp_result[d][u_pair_set][0] + exp_result[d][u_pair_set][2]
        overall_acc[1] += exp_result[d][u_pair_set][1] + exp_result[d][u_pair_set][3]
        if exp_result[d][u_pair_set][1] > 0:
            exp_result[d][u_pair_set].append((exp_result[d][u_pair_set][0] + 
                    exp_result[d][u_pair_set][2]) / (exp_result[d][u_pair_set][1] + exp_result[d][u_pair_set][3]))
        if exp_result[d][u_pair_set][1] > 0:
            exp_result[d][u_pair_set][0] = exp_result[d][u_pair_set][0] / exp_result[d][u_pair_set][1]
            exp_result[d][u_pair_set][4] = exp_result[d][u_pair_set][4] / exp_result[d][u_pair_set][1]
        if exp_result[d][u_pair_set][3] > 0:
            exp_result[d][u_pair_set][2] = exp_result[d][u_pair_set][2] / exp_result[d][u_pair_set][3]
        print(u_pair, exp_result[d][u_pair_set][0], exp_result[d][u_pair_set][2])
print("The overall accuracy is " + str(overall_acc[0]/overall_acc[1]))

avg_gt = sum(gt_probs) / len(gt_probs)
errors = [abs(x-avg_gt) for x in gt_probs]
print("Baseline approach, optimal single prediction: " + str(sum(errors) / len(errors)))
print(max_p)
print(all_state_cnt)

('HS3301', 'HS3309') 0.13026894994926538 0
('HS3301', 'HS5309') 0.129230319761139 0
('HS3309', 'HS5309') 0.12265916260397064 0.31208877729045603
The overall accuracy is 0.13648899609366474
Baseline approach, optimal single prediction: 0.18608134283965502
0.9090909090909091
5744.0


In [9]:
from refit_alpha_test import *

ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            THERMO_MODE_CTX: {},
            "OutTemp#NUM": {
                "range": (-25, 35),
                "interval": 5,
            },
        })
gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            THERMO_MODE_CTX: {},
            "OutTemp#NUM": {
                "range": (-25, 35),
                "interval": 5,
            },
        })


def alpha_nonLinear_generator(alpha_min, alpha_max, alpha_step):
    auc_values = []
    alpha = alpha_min
    while alpha <= alpha_max:
        yield alpha
        if alpha_step <= alpha / 10:
            alpha_step *= 10
        alpha += alpha_step

print("Run test accuracy for context " + str(ctx_info.get_all_ctx_ordered()))
alpha_generator = alpha_nonLinear_generator(1e-6, 0.5, 1e-6)
acc_values = test_acc_alpha(
    ctx_info = ctx_info, 
    gt_ctx_info = gt_ctx_info, 
    root_folder = root_folder,
    alpha_generator = alpha_generator, 
    device_mapping=setpoint_mapping)


Run test accuracy for context ['min_of_day#NUM', 'thermo_mode#CAT', 'OutTemp#NUM']


TypeError: '<=' not supported between instances of 'str' and 'datetime.datetime'